<a href="https://colab.research.google.com/github/ericslevenson/arctic-surface-water/blob/main/dilate2vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
author: ericslevenson
date: 10/16/2022
description: dilate lake predictions and vectorize

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth 
auth.authenticate_user()
# Google Drive setup
from google.colab import drive
drive.mount('/content/drive')
# Complete the environment
!pip install rasterio
!pip install geopandas

In [ ]:
import os
import rasterio
import geopandas
import scipy
import pandas as pd
import numpy as np
from rasterio.mask import mask
from rasterio import plot
from rasterio.merge import merge
from rasterio.plot import show
from rasterio import features
from geopandas import GeoDataFrame
from pandas import DataFrame
from rasterio.features import shapes
from shapely.geometry import shape

In [ ]:
from scipy import ndimage

In [ ]:
predPath = '/content/drive/MyDrive/UNET_outputs/' # path to unet prediction directory
outPath = '/content/drive/MyDrive/UNET_pred_dilated/' # path to dilated prediction directory


In [ ]:
count = 0
for f in os.listdir(predPath):
  count += 1

In [ ]:
# Option 1: Dilate lake predictions, vectorize, export shapefile
kernel = np.ones((3,3),np.uint8) # define kernel matrix for dilation
for i, f in enumerate(os.listdir(predPath)):
    # Read in image to np array
    with rasterio.open(predPath+f) as src:
      data = src.read(1)
      meta = src.meta
      meta.update({'nodata':np.nan})
      trans = src.transform
      #check if binary
      #if data is not binary:
      data = np.where(data > 0.33, 1, 0) # create binary
      dilated_pred = scipy.ndimage.binary_dilation(data, kernel, iterations = 6).astype(np.int32)
      # generate shapes
      shape_gen = ((shape(s), v) for s, v in shapes(dilated_pred, transform=src.transform))
      # build a pd.DataFrame of shapes and convert to geodataframe
      df = DataFrame(shape_gen, columns=['geometry', 'class'])
      gdf = GeoDataFrame(df["class"], geometry=df.geometry, crs=src.crs)
      # filter out non-lake polygons
      gdf = gdf[gdf['class'] == 1]
      # write to file
      gdf.to_file(outPath + f.split('.')[0] + '_bufferedLakes.shp')
    print('writing ' + f + ' aka number ' + str(i+1) + ' of ' + str(count)) 

writing 20200707_06WWC__unet_pred.tif aka number 1 of 1
